**CSCE 421 Final Project: Chess Position Evaluation**

Problem formulation: How can we train a computer to learn the game of chess (or more generally, any board game) with as little human interference (or hard-coded knowledge beyond the game’s rules) proficiently enough to outperform its human (and hard-coded engine) counterparts?

In [1]:
# import useful libraries:
import chess # python-chess library
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# define hyperparameters:
num_epochs = 2 # number of times we iterate over the entire training dataset
batch_size = 4 # number of examples per training iteration
test_size = 0.2 # size of test set
NUM_POSITIONS = 1000000 # number of positions to extract & parse from the input dataset (0 for all)
INPUT_FILE_START_INDEX = 0 # which line in the input file do we start importing positions from?
learning_rate = 0.0001 # the learning rate of the network
optimizer_momentum = 0.9 # the momentum parameter of the optimizer object
MODEL_FILEPATH = 'drive/My Drive/Colab Notebooks/CSCE_421_Final/trained_model.pt' # the filepath of the saved model
LOAD_MODEL = True # True to load the trained model from file, False to make one from scratch
SAVE_MODEL = True # True to save the model after training

**Loading the Data**: 

We import the positions dataset, containing FEN-strings (a popular way to encode chess positions) and their respective engine evaluation in centipawns (a popular computer-chess method for evaluating positions). Evaluations were created using the Stockfish chess engine (https://stockfishchess.org/), one of the world's best chess engines, while looking 22 moves ahead.

In [6]:
# mount Google Drive (for input file)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# open input file:
f = open('drive/My Drive/Colab Notebooks/CSCE_421_Final/chessData.csv')

# ignore first line (header) in the CSV file:
f.readline()

In [ ]:
# skip the file pointer to the start index:
for i in range(INPUT_FILE_START_INDEX):
  f.readline()

We define a utility function to convert a line of input (from the CSV file) to tensor input.

In [3]:
# ----- Utility function to convert FEN string to bitboard -----

# global arrays to avoid redundant code:
PIECES = [chess.PAWN, chess.ROOK, chess.KNIGHT, chess.BISHOP, chess.QUEEN, chess.KING]
COLORS = [chess.WHITE, chess.BLACK]

def FEN_to_bitboard(position, return_float=False):
  FEN, score = position.split(',')

  if score.startswith('#+'):
      score = '20000'
  elif score.startswith('#-'):
      score = '-20000'

  # import the FEN to a board object:
  board = chess.Board(FEN)

  # get the int values of each piece and store in bitboard list:
  bitboard = []
  for color in COLORS:
    for piece in PIECES:
      # get the int value of the piece:
      piece_int = int(board.pieces(piece, color))

      # convert the piece int value to a 64-bit binary string:
      piece_bin = [int(i) for i in format(piece_int, '064b')]
      # reshape the 64-bit array into 2d 8x8 array:
      piece_bin = np.reshape(piece_bin, (-1, 8))

      # add these as array indices to the bitboard:
      bitboard.append(piece_bin)

  # if return_float is True, return a FloatTensor of the board
  # otherwise, save memory with ByteTensor:
  if return_float:
    return torch.FloatTensor(bitboard), torch.FloatTensor([int(score)])
  else:
    return torch.ByteTensor(bitboard), torch.ByteTensor([int(score)])

Define a function to load positions (as bitboards) from the CSV file:

In [ ]:
# create the mini-batch data loader (load n data points from filestream f):
def get_data(filestream, n=1000, PRINT_INDICATOR=100000):
  data = []
  count = 0

  if n == 0:
    for line in filestream.readlines():
      count += 1
      try:
        data.append(FEN_to_bitboard(line))
      except:
        # undo the counter:
        count -= 1
      if count % PRINT_INDICATOR == 0: print(count)
  else:
    for i in range(n):
      try:
        data.append(FEN_to_bitboard(filestream.readline()))
        count += 1
      except: pass
      if count % PRINT_INDICATOR == 0: print(count)
  return data

In [ ]:
# inport and parse the data from the CSV file (only around 5,000,000 datapoints will fit in memory):
data = get_data(f, NUM_POSITIONS)

100000
200000
300000
400000
500000
600000
700000
800000
900000


In [ ]:
# split data into training and testing sets:
len_traindata = int((1 - test_size) * NUM_POSITIONS)
len_testdata = NUM_POSITIONS - len_traindata

train_data = data[0:len_traindata]
test_data = data[len_traindata:]

In [ ]:
# create the PyTorch data loaders:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)

**Creating the Neural Network**

Architecture: a 5-layer convolutional neural network, consisting of 2 convolutional layers and 3 fully-connected layers.

In [4]:
# create a convolutional neural network:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    #input shape (batch_size, 12, 8, 8)
    self.conv1 = nn.Sequential(
                    nn.Conv2d(in_channels=12, out_channels=500, kernel_size=3, padding=2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2)
                 )
    #output shape (batch_size, 500, 5, 5)
    
    self.conv2 = nn.Sequential(nn.Conv2d(500, 250, 2, 1, 2), 
                    nn.ReLU(),
                    nn.MaxPool2d(2)
                 )
    #output shape (batch_size, 250, 4, 4)
    
    self.fc1 = nn.Linear(250*4*4, 100)
    self.fc2 = nn.Linear(100, 20)
    self.out = nn.Linear(20, 1)

  def forward(self, x):
    # apply convolutional layers:
    x = self.conv1(x)
    x = self.conv2(x)

    #Flatten
    x = x.view(-1, 250*4*4)

    # apply linear layers:
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.out(x)
    return x

In [7]:
## Instantiate the network:
net = CNN()
if LOAD_MODEL:
  net.load_state_dict(torch.load(MODEL_FILEPATH))

In [8]:
# define a loss function and an optimizer:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=optimizer_momentum)

In [9]:
# define a train function to train the network:
def train(net, num_epochs, PRINT_ITERATIONS=100000, RECORD_ITERATIONS=1000):
  losses = []
  for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0): 
      # convert inputs to float (on-the-spot for saving memory):
      inputs = inputs.type(torch.FloatTensor)
      labels = labels.type(torch.FloatTensor)

      # zero the parameter gradients:
      optimizer.zero_grad()

      # feedforward:
      outputs = net(inputs)
      # print(outputs)
      # break
      loss = criterion(outputs, labels)

      # backpropagation & optimization:
      loss.backward()
      optimizer.step()

      # record loss every RECORD_ITERATIONS mini-batches
      if np.isnan(loss.item()):
        print(i)
        return losses

      running_loss += loss.item()
      if (i+1) % RECORD_ITERATIONS == 0:
        losses.append(running_loss / PRINT_ITERATIONS)
        running_loss = 0.0

      # print update every PRINT_ITERATIONS mini-batches
      if (i+1) % PRINT_ITERATIONS == 0:
        print('\t%d iterations so far.' % (i+1))
    
    print('---------- EPOCH #%d FINISHED ----------' % epoch)
    # save the model after every epoch (just in case):
    if SAVE_MODEL:
      torch.save(net.state_dict(), MODEL_FILEPATH)

  print('Finished Training')
  return losses

In [ ]:
# train the network:
losses = train(net, num_epochs)

In [ ]:
# function to plot loss over training:
def plot_loss(losses):
  plt.plot(range(len(losses)), losses)
  plt.title('Training Loss')
  plt.xlabel('Iteration (1k training batches)')
  plt.ylabel('Loss (MSE)')
  plt.show()

In [ ]:
# save the model:
if SAVE_MODEL:
  torch.save(net.state_dict(), MODEL_FILEPATH)
  print('New/Updated model saved to %s' % MODEL_FILEPATH)

**Creating the Chess Engine**

The chess engine consists of the trained network and uses minimax with alpha-beta pruning to evaluate moves.

In [19]:
# a function to evaluate a position (done here for clarity):
MATERIAL_SCORES = [100, 500, 275, 325, 900, 10000, -100, -500, -275, -325, -900, -10000]
def evaluate_position(board, add_material_score=False):
  # convert board to bitboard of type tensor.FloatTensor:
  board = FEN_to_bitboard(board.fen() + ", 0", return_float=True)[0]

  # add extra dimension:
  board = board[None]

  # evaluate:
  evaluation = net(board) # the network's evaluation

  if add_material_score:
    # calculate material score:
    material = 0
    for i, piece_score in enumerate(MATERIAL_SCORES):
      material += sum(sum(board[0][i])) * piece_score
    evaluation += material

  return evaluation.item()

In [56]:
# the decision-making function. minimax with alpha-beta pruning:
INF = 1e9
WHITE = True
BLACK = False

def engine(board, depth, maximizingPlayer):
  return str(alphabeta(board, depth, -INF, INF, maximizingPlayer)[1])

def alphabeta(board, depth, alpha, beta, maximizingPlayer):
  if depth == 0 or board.is_game_over():
    return evaluate_position(board, False), None # 'None' is just a placeholder
  if maximizingPlayer:
    value = -INF
    bestmove = None
    for move in board.legal_moves:
      board.push(move) # make the move
      move_score = alphabeta(board, depth-1, alpha, beta, False)[0]
      if move_score > value:
        value = move_score
        bestmove = move
      alpha = max(alpha, value)
      if alpha >= beta:
        break # beta cutoff
      board.pop() # undo the move
    return value, bestmove
  else:
    value = INF
    bestmove = None
    for move in board.legal_moves:
      board.push(move) # make the move
      move_score = alphabeta(board, depth-1, alpha, beta, False)[0]
      if move_score < value:
        value = move_score
        bestmove = move
      beta = min(beta, value)
      if beta <= alpha:
        break # alpha cutoff
      board.pop() # undo the move
    return value, bestmove

Below is an example of the neural network deciding the best move in the given position:

In [55]:
FEN = "r1b3k1/pp1nb1pp/4p3/3pP3/3q4/P2B1P2/1PQB3P/R3K2R w KQ - 0 17"
board = chess.Board(FEN)
engine(board, 3, WHITE)

'd3h7'